In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

# 1) 대상 URL
page = 1
url = f"https://auto.danawa.com/news/?Work=list&Tab=A&Stab=all&Brand=&Model=&HeadText=&NewsGroup=&SearchKey=&SearchWord=&Punit=10&useOldData=&Page={page}"

# 2) 브라우저 위장
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

# 3) HTML 가져오기
req = Request(url, headers=HEADERS)
html = urlopen(req)
soup = BeautifulSoup(html, "html.parser")

# 4) 제목 CSS 선택자
TITLE_SELECTOR = "td.contents div.title a"

# 5) 제목 추출
titles = [tag.get_text(strip=True) for tag in soup.select(TITLE_SELECTOR)]

print("추출된 제목 개수:", len(titles))
for t in titles:
    print("-", t)
